<a href="https://colab.research.google.com/github/mohyunyang/MyStudy/blob/master/solo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

In [29]:
print(val.head(1))
print("===============")
print(test.head(1))

   ID        V1        V2        V3        V4        V5        V6        V7  \
0  10 -0.338262  1.119593  1.044367 -0.222187  0.499361 -0.246761  0.651583   

         V8        V9  ...       V22       V23      V24       V25       V26  \
0  0.069539 -0.736727  ... -0.633753 -0.120794 -0.38505 -0.069733  0.094199   

        V27       V28       V29       V30  Class  
0  0.246219  0.083076 -0.255991 -0.994878      0  

[1 rows x 32 columns]
        ID        V1        V2        V3        V4        V5        V6  \
0  AAAA0x1 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388   

         V7        V8        V9  ...       V21       V22       V23       V24  \
0  0.239599  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928   

        V25       V26       V27       V28       V29       V30  
0  0.128539 -0.189115  0.133558 -0.021053  1.783274 -0.994983  

[1 rows x 31 columns]


# VIF 

In [34]:
# ID 제거
val_y = val['Class']
val = val.drop(['ID','Class'], axis = 1)
test = test.drop('ID', axis = 1)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(val.values,i) for i in range(val.shape[1])]
vif['feature'] = val.columns
vif

# corr()

In [ ]:
plt.figure(figsize=(30,30))
data = val.corr()
sns.heatmap(data, annot=True, fmt='.2f')

# PCA

In [55]:
# # v29 와 v20 
pca1 = val[['V29','V20']]
pca1.head()

,V29,V20
0,-0.255991,0.203711
1,0.168937,-0.269321
2,0.169496,-0.191050
3,0.331307,-0.185390
4,-0.287012,2.456724


In [58]:
# V6 과 V29
pca2 = val[['V6','V29']]
pca2.head(1)

,V6,V29
0,-0.246761,-0.255991


In [60]:
V29 = val['V29']
V30 = val['V30']
print(V29.head(1))
print(V30.head(1))

0   -0.255991
Name: V29, dtype: float64
0   -0.994878
Name: V30, dtype: float64


In [63]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca1 = pca.fit_transform(pca1)
pca1 = pd.DataFrame(pca1)
pca1.columns = ['PCA_1']

pca2 = pca.fit_transform(pca2)
pca2 = pd.DataFrame(pca2)
pca2.columns = ['PCA_2']

,PCA_1
0,-1.152786
1,-0.779088
2,-0.770433
3,-0.608905
4,-0.950500
...,...
28457,-0.716853
28458,-0.110575
28459,-0.405203
28460,-1.214351


In [65]:
val_A = pd.concat([pca1, pca2, V29],axis=1)
val_B = pd.concat([pca1, pca2, V30],axis=1)

print(val_A.head(1))
print(val_B.head(1))

      PCA_1     PCA_2       V29
0 -1.152786 -1.194782 -0.255991
      PCA_1     PCA_2       V30
0 -1.152786 -1.194782 -0.994878


In [72]:
def get_pred_label(model_pred): # val_pred 이 들어간다면??  # val_pred = m.predict(val_pca) 즉, "model_rc로 차원축소한 val 데이터"를 넣는다면? # val_pred 는 차원축소한 val데이터를 rc모델로 예측한 값들이 들어가 있다.
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)  # np.where() 은 조건을 만족하는 위치의 인덱스를 가져옴
    model_pred = np.where(model_pred == -1, 1, model_pred) # np.where(a조건,b,c) a조건이면 b, 그렇지 않으면 c의 인덱스를 가져옴 
    return model_pred

In [74]:
from sklearn.covariance import EllipticEnvelope
contamination_controlnumber=0
model_rc = EllipticEnvelope(contamination=outliers_fraction + 
                            contamination_controlnumber)
model_rc.fit(val_A)

val_pred = m.predict(val_A)
val_pred = get_pred_label(val_pred)

val_pred

NameError: ignored